# Praktiskā nodarbība: Teksta izguve | Hands-on session: Text extraction

Teksta korpusa izveide ir viens no priekšnoteikumiem daudzos valodas modelēšanas uzdevumos. Tā kā vienkāršs teksts (*plain-text*) bieži vien nav pieejams, nākas nodarboties ar vienkārša teksta izguvi no dokumentiem, kas pieejami citos formātos.

Šajā piezīmju grāmatiņā ir apskatīti trīs vienkāršoti gadījumi teksta izguvei no dažāda formāta dokumentiem: HTML, PDF un VERT.

Populārajiem dokumentu formātiem (PDF, DOCX, HTML u.c.) eksistē dažādas *Python* bibliotēkas, kuras var izmantot vienkāršā teksta izguves uzdevumam, kā tas ir nodemonstrēts šajā nodarbībā.

---

Creating a text corpus is one of the prerequisites for many language modeling tasks. Since plain-text is often not available, one has to deal with plain-text extraction from documents that are available in other formats.

This notebook covers three simplified cases for extracting text from documents in various formats: HTML, PDF and VERT.

For the popular document formats (PDF, DOCX, HTML, etc.), there are various Python libraries that can be used for the plain-text extraction task as demonstrated in this session.

### HTML-to-Text

Viena no populārākajām un vienkāršāk izmantojamajām *Python* bibliotēkām HTML formāta dokumentu parsēšanai un satura "noskrāpēšanai" (*web scraping*) ir *BeautifulSoup* (https://pypi.org/project/beautifulsoup4/).

*BeautifulSoup* savukārt izmanto zemāka līmeņa HTML/XML parsētāju: var tikt izmantots gan *Python* iebūvētais `html.parser`, gan ārējas bibliotēkas (piem., `lxml`, `html5lib`), kas nodrošina dažādas priekšrocības (piem., ātrdarbību un papildu funkcionalitāti). Šajā demonstrācijā ir izmantots iebūvētais HTML parsētājs.

In [6]:
!pip install beautifulsoup4

In [2]:
!wget -O "sample_article.html" https://www.vti.lu.lv/par-mums/zinas/zina/t/82316/

--2024-02-14 10:07:43--  https://www.vti.lu.lv/par-mums/zinas/zina/t/82316/
Resolving www.vti.lu.lv (www.vti.lu.lv)... 5.179.1.160
Connecting to www.vti.lu.lv (www.vti.lu.lv)|5.179.1.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28753 (28K) [text/html]
Saving to: ‘sample_article.html’

sample_article.html 100%[===================>]  28.08K  --.-KB/s    in 0.1s    

2024-02-14 10:07:44 (212 KB/s) - ‘sample_article.html’ saved [28753/28753]



In [30]:
from bs4 import BeautifulSoup
import html
import re


# Funkcija veic noteikta veida HTML elementu izmešanu, lai atbīvotos no nelietderīgiem teksta blokiem.
# Piemēri: izvēlnes, kājenes, galvenes, tabulas, kas satur tikai skaitliskus datus vai izolētus vārdus.
# Funkciju nepieciešams pielāgot konkrēto vietņu īpatnībām, lai iegūtu iespējami kvatitatīvu teksta saturu.
def remove_html_elements(text):
    soup = BeautifulSoup(text, "html.parser")

    # Meklēšana pēc HTML elementa
    for element in soup.find_all(["header", "footer", "button"]):
        element.decompose()

    # Meklēšana pēc HTML elementa ar noteiktu atribūtu
    for element in soup.find_all(["div"], attrs={"class": re.compile(".*([Mm]enu|share|backlink).*")}):
        element.decompose()

    return str(soup)

# Funkcija veic (1) HTML entitāšu konvertēšanu (escaped=>unescaped) un (2) HTML birku izmešanu (paturot elementu saturu).
# Piemēri: &amp; => &, <i>vārds</i> => vārds
# Šī funkcija ir universāla - pielietojama jebkuras vietnes satura normalizēšanai.
def convert_html_entities(text):
    text = html.unescape(text)                      # 1
    text = BeautifulSoup(text, "html.parser").text  # 2
    return text

# Funkcija veic atstarpju un tukšo rindu normalizāciju iegūtajā vienkāršajā tekstā.
def normalize_white_spaces(text):
    text = re.sub("[ ]+", " ", text)
    text = re.sub("[ ]?\n+", "\n", text)
    return text

# Funkcija veic (1) lieko HTML elementu izmešanu, (2) HTML entitāšu normalizēšanu un birku izmešanu, (3) atstarpju normalizēšanu.
def html_to_txt(html_file, txt_file):
    input_file = open(html_file, "r", encoding="utf-8")
    output_file = open(txt_file, "w", encoding="utf-8")

    text = input_file.read()
    text = remove_html_elements(text)   # 1
    text = convert_html_entities(text)  # 2
    text = normalize_white_spaces(text) # 3
    output_file.write(text)

    input_file.close()
    output_file.close()


html_to_txt("sample_article.html", "sample_article.txt")

### PDF-to-Text

Saistīta teksta izguve no PDF dokumentiem lielākoties ir sarežģīta un ķēpīga: informācija par teksta struktūru un noformējumu bieži vien nav viennozīmīgi izgūstama, un teksta segmentēšana teikumos un rindkopās ir apgrūtināta, jo PDF formāts ir veidots satura drukāšanas nevis mašīnlasīšanas vajadzībām.

Arī PDF dokumentu apstrādei ir pieejamas dažādas ārējās *Python* bibliotēkas, piemēram, `pypdf`, `PyPDF2`, `PDFMiner`, `PyMuPDF`, `tabula-py`. Demonstrācijas nolūkiem izmantosim `pypdf` (https://pypi.org/project/pypdf/).

Plašāk par PDF dokumentu mašīnlasīšanas problemātiku aprakstīts `pypdf` dokumentācijā: https://pypdf.readthedocs.io/en/stable/user/extract-text.html

Eksperimentēšanas vērta alternatīva pieeja: konvertēt PDF uz HTML un tālāk strādāt ar HTML dokumentiem. Konvertēšanas funkcionalitāti nodrošina, piemēram, `PDFMiner` (https://pypi.org/project/pdfminer/).

In [31]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 2.6 MB/s eta 0:00:00


In [32]:
!wget -O "sample_paper.pdf" https://www.apgads.lu.lv/fileadmin/user_upload/lu_portal/apgads/PDF/Valoda-nozime-forma/VNF-10/vnf_10-16_Nespore_Saulite_Rituma.pdf

--2024-02-14 12:06:20--  https://www.apgads.lu.lv/fileadmin/user_upload/lu_portal/apgads/PDF/Valoda-nozime-forma/VNF-10/vnf_10-16_Nespore_Saulite_Rituma.pdf
Resolving www.apgads.lu.lv (www.apgads.lu.lv)... 5.179.1.160
Connecting to www.apgads.lu.lv (www.apgads.lu.lv)|5.179.1.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 920996 (899K) [application/pdf]
Saving to: ‘sample_paper.pdf’

sample_paper.pdf    100%[===================>] 899.41K  1.10MB/s    in 0.8s    

2024-02-14 12:06:22 (1.10 MB/s) - ‘sample_paper.pdf’ saved [920996/920996]



In [43]:
from pypdf import PdfReader


def pdf_to_txt(pdf_file, txt_file):
    input_file = open(pdf_file, "rb")
    output_file = open(txt_file, "w", encoding="utf-8")

    reader = PdfReader(input_file)
    text = ""

    # Nolasa dokumentu pa vienai lapai.
    for page in reader.pages:
        text += page.extract_text() + "\n"

    output_file.write(normalize_white_spaces(text))
    output_file.close()


pdf_to_txt("sample_paper.pdf", "sample_paper.txt")

Papētot iegūto rezultātu (`sample_paper.txt`), redzams, ka ar tik vienkāršiem soļiem nepietiek, lai no PDF dokumenta izgūtu kvalitatīvu tekstu.

Izteiktākā problēma ir tā, ka izgūtajā tekstā ir saglabāts teksta dalījums rindās un lappusēs tā, kā tas drukas vajadzībām ir izkārtots PDF dokumentā, bet mums ir nepieciešams teksts, kas būtu strukturēts atbilstoši teikumu un rindkopu robežām, nevis dokumenta vizuālajam noformējumam.

Lai iegūtu vēlamo rezultātu (t.i., tuvinātos tam), ir jāveic papildu soļi teksta noformējuma analīzē un atbilstošā pēcapstrādē.

In [45]:
def merge_lines(text):
    # Ja rinda beidzas ar burtu un defisi, pieņemam, ka tas ir vārda pārnesums jaunā rindā.
    text = re.sub(r"(?<=[a-zāčēģīķļņšūž])[--]\n(?=[a-zāčēģīķļņšūž])", "", text)

    # Ja rinda sākas ar mazo burtu, pieņemam, ka teikums turpinās.
    text = re.sub(r"(\n)+(?=[a-zāčēģīķļņšūž])", " ", text) # FIXME: \p{Ll}

    return text

def pdf_to_txt_2(pdf_file, txt_file):
    input_file = open(pdf_file, "rb")
    output_file = open(txt_file, "w", encoding="utf-8")

    reader = PdfReader(input_file)
    text = ""

    for page in reader.pages:
        text += page.extract_text() + "\n"

    text = merge_lines(normalize_white_spaces(text))

    output_file.write(text)
    output_file.close()


pdf_to_txt_2("sample_paper.pdf", "sample_paper_2.txt")

### VERT-to-Text

Dažkārt nākas saskarties ar failu formātiem, kas nav plaši izplatīti vai tiek izmantoti pamatā tikai valodu tehnoloģiju jomā, un to apstrādei nav pieejamas jau gatavas bibliotēkas, vai arī pastāv vairāki varianti, kā attiecīgais datu formāts var tikt realizēts vai interpretēts.

Daži piemēri: CoNLL, VERT, TSV3 - specifiski *tab-separated* failu formāti, kas tiek izmantoti valodu tehnoloģiju jomā. Šādos gadījumos nepieciešams analizēt faila struktūru un attiecīgi iegūt un pārveidot tikai nepieciešamo informāciju.

Demonstrācijā aplūkosim VERT/VRT formātu, kura dažādus atvasinājumus izmanto teksta korpusu platformas [SketchEngine](https://www.sketchengine.eu/my_keywords/vertical/), NoSketchEngine, [Korp](https://www.kielipankki.fi/development/korp/corpus-input-format/) u.c. Šis formāts tiek izmantots arī latviešu valodas korpusu platformā [Korpuss.lv](https://korpuss.lv/), kas izmanto NoSketchEngine.

Lai no VERT formāta iegūtu vienkāršu, saistītu tekstu ir nepieciešams ievērot teksta segmentēšanu teikumos un teikumu segmentēšanu tekstvienībās atbilstoši VERT formātā lietotajam strukturālajam marķējumam.

Demonstrācijai izmantosim Raiņa korpusu, kas atrodams CLARIN-LV repozitorijā: http://hdl.handle.net/20.500.12574/41

In [2]:
!wget -O "sample_corpus.vert" https://repository.clarin.lv/repository/xmlui/bitstream/handle/20.500.12574/41/rainis_v20180716.vert

--2024-02-15 10:15:43--  https://repository.clarin.lv/repository/xmlui/bitstream/handle/20.500.12574/41/rainis_v20180716.vert
Resolving repository.clarin.lv (repository.clarin.lv)... 92.240.80.87
Connecting to repository.clarin.lv (repository.clarin.lv)|92.240.80.87|:443... connected.
HTTP request sent, awaiting response... 200 200
Length: 47156135 (45M) [application/octet-stream]
Saving to: ‘sample_corpus.vert’

sample_corpus.vert  100%[===================>]  44.97M  14.0MB/s    in 3.5s    

2024-02-15 10:15:48 (12.8 MB/s) - ‘sample_corpus.vert’ saved [47156135/47156135]



In [4]:
def vert_to_txt(vert_file, txt_file):
    input_file = open(vert_file, "r", encoding="utf-8")
    output_file = open(txt_file, "w", encoding="utf-8")
    text = ""

    # Nolasa VERT failu pa vienai rindiņai un rekonstruē oriģinālos teikumus un to dalījumu rindkopās.
    while True:
        line = input_file.readline()

        if not line: break

        if line == "\n":
            if text != "":
                output_file.write(text + "\n")
            text = ""

        # Ja tiek nolasīts marķējuma simbols, veicam attiecīgās struktūras apstrādi.
        elif line[0] == "<" and line[1] != "\t":

            # </doc>, </p>, </s> - apzīmē dokumenta, paragrāfa, teikuma beigas.
            # Līdz šim atmiņā rekonstruēto teikumu ieraksta izvadfailā un sāk nākamā teikuma rekonstruēšanu.
            if line == "</doc>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                output_file.write(text + "\n\n")
                text = ""
            elif line == "</p>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                output_file.write(text + "\n")
                text = ""
            elif line == "</s>\n":
                if text[:-1] == " ":
                    text = text[:-1]
                output_file.write(text)
                text = ""

            # <g/> - "glue tag" norāda, ka starp tekstvienībām nav jabūt atstarpei.
            # Piemērs: vārds un tam sekojoša interpunkcijas zīme.
            elif line == "<g />\n" and len(text) > 1:
                if text[-1] == " ":
                    text = text[:-1]

            # Ignorē atverošos <doc>, <p>, <s> marķējumus.
            else:
                continue

        # Ja faila tekošā rindiņa nesatur marķējuma simbolu, tad tā satur tekstvienību - pievienojam to izvadam.
        # Atbilstoši VERT struktūrai, katras šādas rindiņas pirmais elements (lauks) satur tekstvienību tās oriģinālajā formā.
        else:
            text = text + line.split("\t")[0] + " "


vert_to_txt("sample_corpus.vert", "sample_corpus.txt")